In [16]:
# Written by *** for COMP9021
#
# Randomly generates a grid with 0s and 1s, whose dimension is controlled
# by user input, as well as the density of 1s in the grid, and finds out,
# for given step_number >= 1 and step_size >= 2, the number of stairs of
# step_number many steps, with all steps of size step_size.
#
# A stair of 1 step of size 2 is of the form
# 1 1
#   1 1
#
# A stair of 2 steps of size 2 is of the form
# 1 1
#   1 1
#     1 1
#
# A stair of 1 step of size 3 is of the form
# 1 1 1
#     1
#     1 1 1
#
# A stair of 2 steps of size 3 is of the form
# 1 1 1
#     1
#     1 1 1
#         1
#         1 1 1
#
# The output lists the number of stairs from smallest step sizes to largest
# step sizes, and for a given step size, from stairs with the smallest number
# of steps to stairs with the largest number of stairs.


from random import seed, randint
import sys
from collections import defaultdict
import math


def display_grid():
    for i in range(len(grid)):
        print('   ', ' '.join(str(int(grid[i][j] != 0))
                              for j in range(len(grid))
                             )
             )

try:
    arg_for_seed, density, dim = (int(x) for x in 
                        input('Enter three positive integers: ').split()
                                 )
    if arg_for_seed < 0 or density < 0 or dim < 0:
        raise ValueError
except ValueError:
    print('Incorrect input, giving up.')
    sys.exit()
seed(arg_for_seed)
grid = [[randint(0, density) for _ in range(dim)] for _ in range(dim)]
print('Here is the grid that has been generated:')
display_grid()




def stairs_in_grid():
    stairs = {}    
    for i in range(2, math.ceil(dim/2) + 1):
        step_counts = dict(sorted(check_size(i).items(), key=lambda item: item[0]))
    
        if len(step_counts) == 0:
            continue
        stairs[i] = []
        for step in step_counts:
            stairs[i].append([step, step_counts[step]])
    return stairs
    # REPLACE THE RETURN STATEMENT ABOVE WITH YOUR CODE

# POSSIBLY DEFINE OTHER FUNCTIONS
def trace_steps(grid, row, col, size, known_steps):
    # Returns number of steps
    known_steps.append((row, col))
    step_count = 0
    while col + step_count + size < dim and row + (size-1)*step_count + (size-1)*1 < dim: #FIX
        step_count += 1
        start = col + step_count * (size-1)
        stop = start + size
#         print(f'Step = {step_count}, pair = {color_grid[row + (size-1)*step_count][start:stop]}')#FIX
        strip = color_grid[row + (size-1) * step_count][start:stop]      
        if strip.__contains__("⬛") or len(strip) != size:
            step_count -= 1
            break
            
        # Need to check stem first
#         print('checking column between')
        valid = True
        for i in range(size):
#             print(f'{i}:{color_grid[row + (size-1) * (step_count - 1) + i][start]}')
            if color_grid[row + (size-1) * (step_count - 1) + i][start] == "⬛":    
                
                # (size-1) * (step_count - 1) + i
                valid = False
        if valid == False:
            step_count -= 1
            break
            
        # if it passes the two steps then we assess this as a known row 
        known_steps.append((row + (size-1)*step_count, start)) #FIX
#     print(f'Steps at {col,row} = {step_count}')
    return step_count

def append_step_counter(step_counter, step_count):
    if step_counter.__contains__(step_count):
        step_counter[step_count] = step_counter[step_count] + 1
    else:
        step_counter[step_count] = 1

def check_size(size):
    known_steps = []
    step_counter = {}
    step_count = 0
    for row in range(dim-1):
        for col in range(dim-1):
#             print('************************')
#             print(f'Step = 0, pair = {color_grid[row][col:col+size]}, = {color_grid[row][col:col+size].__contains__("⬛")}')
            if not known_steps.__contains__((row, col)) and not color_grid[row][col:col+size].__contains__("⬛"):
                step_count = trace_steps(color_grid, row, col, size, known_steps)   
            if step_count > 0:
                append_step_counter(step_counter, step_count)
                step_count = 0
    return step_counter

# Creating a color grid to more easily process

color_grid = [['' for _ in range(dim)] for _ in range(dim)]
count = 0
for i in range(dim):
    for j in range(dim):
        if grid[i][j] > 0:
#             color_grid[i][j] = '⬜'
            color_grid[i][j] = str(i)
            count += 1
        else:
            color_grid[i][j] = '⬛'  
        

# A dictionary whose keys are step sizes, and whose values are pairs
# of the form (number_of_steps,
#              number_of_stairs_with_that_number_of_steps_of_that_step_size
#             ),
# ordered from smallest to largest number_of_steps.
stairs = stairs_in_grid()
for step_size in sorted(stairs):
    print(f'\nFor steps of size {step_size}, we have:')
    for nb_of_steps, nb_of_stairs in stairs[step_size]:
        stair_or_stairs = 'stair' if nb_of_stairs == 1 else 'stairs'
        step_or_steps = 'step' if nb_of_steps == 1 else 'steps'
        print('    ', nb_of_stairs, stair_or_stairs, 'with',
              nb_of_steps, step_or_steps
             )

Enter three positive integers: 0 4 8
Here is the grid that has been generated:
    1 1 0 1 1 1 1 1
    1 1 1 1 1 1 1 1
    0 1 1 1 1 1 1 0
    0 1 1 1 0 1 1 1
    1 1 1 1 1 1 1 0
    1 0 0 1 0 1 1 1
    1 1 0 1 1 1 1 1
    1 1 0 0 1 1 1 0
************************
Step = 0, pair = ['0', '0'], = False
Step = 1, pair = ['1', '1']
checking column between
0:0
1:1
Step = 2, pair = ['2', '2']
checking column between
0:1
1:2
Step = 3, pair = ['3', '⬛']
Steps at (0, 0) = 2
************************
Step = 0, pair = ['0', '⬛'], = True
************************
Step = 0, pair = ['⬛', '0'], = True
************************
Step = 0, pair = ['0', '0'], = False
Step = 1, pair = ['1', '1']
checking column between
0:0
1:1
Step = 2, pair = ['2', '2']
checking column between
0:1
1:2
Step = 3, pair = ['3', '3']
checking column between
0:2
1:3
Steps at (3, 0) = 3
************************
Step = 0, pair = ['0', '0'], = False
Step = 1, pair = ['1', '1']
checking column between
0:0
1:1
Step = 2, pair = ['2', '⬛

In [35]:
# stairs = stairs_in_grid()
# find "Steps at (3, 4)"

